In [52]:
import numpy as np
import math
from numpy import *

In [53]:
#algoritmo para predecir la posicion en el espacio

In [88]:
#algoritmo de Newtom Rhapson
#f,df son la funcion y su derivada
#Vin y N son el estimativo inicial y el numero de iteraciones 
def nr(f,df,Vin,N):
    p=Vin
    for i in xrange(0,N):
        p=p-(f(p)/df(p))
    return p

In [89]:
DIA=86400.0 # s
DEG=pi/180
RAD=180/pi
AU=1.5e11 # m
def Esol(M,e):
    """
    Mikkola, 1991
    Code at: http://smallsats.org/2013/04/20/keplers-equation-iterative-and-non-iterative-solver-comparison/
    """
    Ecorr=0;Esgn=1.0
    if M>180*DEG:
        M=360.0*DEG-M
        Ecorr=360*DEG
        Esgn=-1.0
    if e==0:return Ecorr+Esgn*M
        
    a=(1-e)*3/(4*e+0.5);
    b=-M/(4*e+0.5);
    y=(b**2/4 +a**3/27)**0.5;
    x=(-0.5*b+y)**(1./3)-(0.5*b+y)**(1./3);
    w=x-0.078*x**5/(1 + e);
    E=M+e*(3*w-4*w**3);

    #NEWTON CORRECTION 1
    sE=sin(E)
    cE=cos(E)

    f=(E-e*sE-M);
    fd=1-e*cE;
    f2d=e*sE;
    f3d=-e*cE;
    f4d=e*sE;
    E=E-f/fd*(1+\
                  f*f2d/(2*fd*fd)+\
                  f*f*(3*f2d*f2d-fd*f3d)/(6*fd**4)+\
                  (10*fd*f2d*f3d-15*f2d**3-fd**2*f4d)*\
                  f**3/(24*fd**6))

    #NEWTON CORRECTION 2
    f=(E-e*sE-M);
    fd=1-e*cE;
    f2d=e*sE;
    f3d=-e*cE;
    f4d=e*sE;
    E=E-f/fd*(1+\
                  f*f2d/(2*fd*fd)+\
                  f*f*(3*f2d*f2d-fd*f3d)/(6*fd**4)+\
                  (10*fd*f2d*f3d-15*f2d**3-fd**2*f4d)*\
                  f**3/(24*fd**6))
    
    E=Ecorr+Esgn*E
    return E

In [90]:
#Ecuacion de Kepler
#los valores de M,E se dan en radianes
def eckepler(E):
    kepler=E-e*np.sin(E)-M
    return kepler
#derivada ecuacion de kepler
def deckepler(E):
    dkepler=1-e*np.cos(E)
    return dkepler

In [114]:
#matris de rotacion, angulos en radianes
def rotacion(omega,inc,w):
    rot1=np.array([[np.cos(omega),-np.sin(omega),0.0],
             [np.sin(omega),np.cos(omega),0.0],
             [0.0,0.0,1.0]])
    rot2=np.array([[1.0,0.0,0.0],
               [0.0,np.cos(inc),np.sin(inc)],
               [0.0,-np.sin(inc),np.cos(inc)]])
    rot3=np.array([[np.cos(w),np.sin(w),0.0],
             [-np.sin(w),np.cos(w),0.0],
             [0.0,0.0,1.0]])
    rot_final=(rot3.dot(rot2)).dot(rot1)
    return rot_final

In [92]:
#anomalia verdadera, E en grados
def anomalia_verdadera(E,e):
    factor=((1.0+e)/(1.0-e))**0.5
    f=2.0*np.arctan(factor*np.tan(E/2.0))
    return f

In [93]:
#datos

In [94]:
grad=180.0/np.pi
rad=np.pi/180.0

In [95]:
#unidades canónicas
um=1.988544*10**30.0
ul=149597870700.0
G=6.67392*10**(-11.0)
ut=(ul**3.0/(um*G))**0.5
mu=(G*um)*(ut**2.0/ul**3.0)
dia=ut*(1.0/3600.0)*(1.0/23.9345)
print ut,dia,mu

5022616.48057 58.2912216509 1.0


In [96]:
tp=2457427.543855121499    #fecha juliana, dias,2016-Feb-09.04385512
t=2458459.2916667          #fecha juliana, dias UT-5 2018/02/06 14:00:00
#t=2458459.0833333          #fecha juliana, dias UT    2018/02/06 14:00:00
#t=2457430.0833333          #2016-feb-11 14:00:00
periodo=2047.343285965675  #dias
e=0.1872699504244361       #ecentricidad
a=3.155451391965179        #semieje mayor
#n=2*np.pi/periodo          #mean orbital motion numericamente rad/dia
n=0.1758376342979521*rad   #valor de Horizons rad/dia
M=n*(t-tp)                 #anomalia media  
print 'M =',M*grad

M = 181.42009438


In [104]:
#anomalia ecentrica y verdadera
E=nr(eckepler,deckepler,M,10)
#E=Esol(M,e)
f=1.0*anomalia_verdadera(E,e)
cero=E-e*np.sin(E)-M #verificacion de E, debe dar cero esta linea
print 'E =',E*grad
print 'f =',f*grad
print 'Aqui debe dar cero',cero
print 'Solucion de Jorge = ',Esol(M,e)*grad

E = 181.196114371
f = -179.010362588
Aqui debe dar cero 0.0
Solucion de Jorge =  181.192336672


In [120]:
#posicion y velosidad en el plano de referencia
p=a*(1.0-e**2)                                         #semilatus rectum
v1=-1.0*(mu/p)**0.5                                    #factor para v
r_mag=p/(1.0+e*np.cos(f))                              #magnitud de r con f
#r_mag=a*(1-e*np.cos(E))                                #magnitud de r con E
r=np.array([r_mag*np.cos(f),r_mag*np.sin(f),0.0])      #r vector
vel=np.array([v1*np.sin(f),-1.0*v1*(e+np.cos(f)),0.0]) #v vector
print 'r =',r
print 'v =',vel

r = [-3.74568505 -0.06470352  0.        ]
v = [ 0.00989813 -0.4656804   0.        ]


In [121]:
#elementos orbitales
#tomado de Horizons
omega=110.9390420742303*rad
w=358.7790635893293*rad
inc=2.73824776304453*rad

In [122]:
#matris de rotacion
rot_final=rotacion(omega,inc,w)

In [123]:
r_rot=rot_final.dot(r)
v_rot=rot_final.dot(vel)
print 'r rotado =',r_rot
print 'v rotada =',(1.0/dia)*v_rot  #de ul por ut a ul por dia

r rotado = [ 1.47268941 -3.4406345   0.16602164]
v rotada = [ 0.00733478  0.00316717 -0.00014397]


In [124]:
""""
datos de Horizons:
2458459.083333333 = A.D. 2018-Dec-06 14:00:00.0000 (TDB)
   1.308057620446669E+00 -3.510316245875934E+00  1.194654235809810E-03
   7.457989109089521E-03  2.817550176085307E-03 -3.817166437810427E-04
   2.163572403152746E-02  3.746109490080055E+00 -3.616250114796165E-05
   
2457430.083333333 = A.D. 2016-Feb-11 14:00:00.0000 (TDB)
  -8.894479089857996E-01  2.405670048896956E+00 -1.352903896373736E-03
  -1.096617715140197E-02 -4.044752792298253E-03  5.591054548037377E-04
   1.481323942244966E-02  2.564832898515621E+00  8.868520566378756E-06
"""

'"\ndatos de Horizons:\n2458459.083333333 = A.D. 2018-Dec-06 14:00:00.0000 (TDB)\n   1.308057620446669E+00 -3.510316245875934E+00  1.194654235809810E-03\n   7.457989109089521E-03  2.817550176085307E-03 -3.817166437810427E-04\n   2.163572403152746E-02  3.746109490080055E+00 -3.616250114796165E-05\n   \n2457430.083333333 = A.D. 2016-Feb-11 14:00:00.0000 (TDB)\n  -8.894479089857996E-01  2.405670048896956E+00 -1.352903896373736E-03\n  -1.096617715140197E-02 -4.044752792298253E-03  5.591054548037377E-04\n   1.481323942244966E-02  2.564832898515621E+00  8.868520566378756E-06\n'